In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import faiss
from faiss import write_index, read_index

import numpy as np
import torch
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer,AutoModelForCausalLM


/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import pandas as pd
df = pd.read_csv(r"D:\COMPETITIONS\RetrieverNLP\PTMTuning\input\eedi_train.csv")

ModuleNotFoundError: No module named 'datasets'

In [11]:
print(batch['query']['input_ids'].shape)  # Should be (2, 3)
batch['contents']['input_ids'].shape  # Should be (2, 2, 3) (2 samples, 2 contents each, 3 tokens each)

torch.Size([2, 3])


torch.Size([2, 2, 3])

In [5]:
content_tokens

{'input_ids': tensor([[[ 1,  2,  3],
          [ 4,  5,  6]],
 
         [[ 7,  8,  9],
          [10, 11, 12]]]),
 'attention_mask': tensor([[[1, 1, 1],
          [1, 1, 1]],
 
         [[1, 1, 1],
          [1, 1, 1]]])}

In [2]:
# MODELS_TO_TEST = {
#     "gte-Qwen2-7B-instruct": {
#         "model_name": "Alibaba-NLP/gte-Qwen2-7B-instruct",
#         "query_instruction": "",
#         "normalize": False,
#         "trust_remote_code": True,  # 必须启用
#         "model_class": "AutoModelForCausalLM"  # 指定生成式模型接口
#     }
# }
# model_config = MODELS_TO_TEST['gte-Qwen2-7B-instruct']
# model = AutoModelForCausalLM.from_pretrained(
#         model_config["model_name"],
#         trust_remote_code=True,
#         device_map="auto",
#         torch_dtype=torch.float16
#     )

In [3]:


class CFG:
    input_path = "/root/cloud/RetrieverNLP/PTM_tuning/input/"
    train_path = f"{input_path}train.csv"
    test_path  = f"{input_path}test.csv"
    misc_path  = f"{input_path}misconception_mapping.csv"
    samp_path  = f"{input_path}sample_submission.csv"
#     max_cutoff = 50 #V1
    #max_cutoff = 100 #V2
    #max_cutoff = 50 #v3
    max_cutoff = 150 #v3
    
    is_train   = False
    if is_train:
        embd_name  = "BAAI/bge-large-en-v1.5"#online
        rerank_na  = 'BAAI/bge-reranker-large'#online
    else:
        embd_name  = "/kaggle/input/bge-large-en-v1-5/bge-large-en-v1.5"#offline
        rerank_na  = "/kaggle/input/bge-reranker-large"#offline
        
    with_fineture_reranker = True
    reranker_fineture_path = "/kaggle/input/bge-reranker-ft-v2"
#     reranker_fineture_path = "/kaggle/input/bge-reranker-ft-v3"
cfg = CFG()


train                 = pd.read_csv(cfg.train_path)
misconception_mapping = pd.read_csv(cfg.misc_path)



def make_all_question_text(df: pd.DataFrame) -> pd.DataFrame:
    df["all_question_text"] = df["ConstructName"] +" " +df["QuestionText"]
    return df
train = make_all_question_text(train)


import pandas as pd

def wide_to_long(df: pd.DataFrame) -> pd.DataFrame:
    # 第一步：处理答案文本（AnswerXText）
    text_long = pd.melt(
        df,
        id_vars    = ["QuestionId", "all_question_text", "CorrectAnswer"],
        value_vars = ["AnswerAText", "AnswerBText", "AnswerCText", "AnswerDText"],
        var_name   = 'Answer',
        value_name = 'AnswerText'
    )
    # 提取答案选项字母（如 A/B/C/D）
    text_long['Answer'] = text_long['Answer'].str.replace('Answer', '').str.replace('Text', '')

    # 第二步：处理错误概念ID（MisconceptionXId）
    misconception_long = pd.melt(
        df,
        id_vars    = ["QuestionId"],
        value_vars = ["MisconceptionAId", "MisconceptionBId", "MisconceptionCId", "MisconceptionDId"],
        var_name   = 'MisconceptionAnswer',
        value_name = 'MisconceptionId'
    )
    # 提取答案选项字母（如 A/B/C/D）
    misconception_long['Answer'] = misconception_long['MisconceptionAnswer'].str.replace('Misconception', '').str.replace('Id', '')
    misconception_long = misconception_long.drop(columns=['MisconceptionAnswer'])

    # 合并两个长格式数据
    merged_long = pd.merge(
        text_long,
        misconception_long,
        on=['QuestionId', 'Answer'],
        how='left'
    )
    
    return merged_long

train_long = wide_to_long(train)

def make_all_text(df: pd.DataFrame) -> pd.DataFrame:
    df["all_text"] = df["all_question_text"] +" " +df["Answer"]
    df = df[ (df.CorrectAnswer!=df.Answer)&(df.MisconceptionId.isnull()==False) ].reset_index(drop=True)
    return df

train_long = make_all_text(train_long)









In [11]:

# ======================================================
# 配置区（按需修改）
# ======================================================
	
# bge-en-icl
MODELS_TO_TEST = {

    "BGE-large-en": {
        "model_name": "BAAI/bge-large-en-v1.5",
        "query_instruction": "Represent this sentence for searching relevant passages: ",
        "normalize": True,
        "trust_remote_code":False
    },
    "BGE-M3": {
        "model_name": "BAAI/bge-m3",
        "query_instruction": "",
        "normalize": False , # M3自带归一化
        "trust_remote_code":False
    },

    
     "jina-embeddings-v3": {
        "model_name": "jinaai/jina-embeddings-v3",
        "query_instruction": "",
        "normalize": True , 
        "trust_remote_code":True  
    },
    
    "snowflake-arctic-embed-l-v2.0": {
            "model_name": "Snowflake/snowflake-arctic-embed-l-v2.0",
            "query_instruction": "",
            "normalize": True , 
            "trust_remote_code":False  
        },
    

}

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PER_GPU_BATCH_SIZE = 32

# ======================================================
# 核心评估函数
# ======================================================
import numpy as np
import pandas as pd
import torch
import faiss
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from sklearn.metrics.pairwise import cosine_similarity

# 新版本加载模型代码，解决KeyError: 'mistral'问题
def evaluate_model(model_config, misconception_mapping, train_data):
    """评估单个模型的Recall@50"""
    model_name = model_config["model_name"]
    trust_remote_code = model_config.get("trust_remote_code", False)
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    PER_GPU_BATCH_SIZE = 32
    
    print(f"Loading model: {model_name}")
    
    try:
        # 尝试先判断模型类型
        if "Qwen" in model_name:
            print("识别为Qwen生成模型，使用AutoModelForCausalLM加载")
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                trust_remote_code=trust_remote_code,
                torch_dtype=torch.float16 if "7B" in model_name else torch.float32,  # 大模型使用半精度
                device_map="auto" if torch.cuda.is_available() else None
            )
            tokenizer = AutoTokenizer.from_pretrained(
                model_name,
                trust_remote_code=trust_remote_code
            )
        elif model_name=='snowflake-arctic-embed-l-v2.0':
            print("使用AutoModel加载嵌入模型")
            model = AutoModel.from_pretrained(
                model_name,
                trust_remote_code=trust_remote_code
            )
            tokenizer = AutoTokenizer.from_pretrained(
                model_name,
                add_pooling_layer=False,
                trust_remote_code=trust_remote_code
            )
        else:
            # 嵌入模型默认使用AutoModel
            print("使用AutoModel加载嵌入模型")
            model = AutoModel.from_pretrained(
                model_name,
                trust_remote_code=trust_remote_code
            )
            tokenizer = AutoTokenizer.from_pretrained(
                model_name,
                trust_remote_code=trust_remote_code
            )
        

        model = model.to(DEVICE)
        print(f"模型加载成功: {model_name}")
        
    except KeyError as e:
        if "mistral" in str(e):
            print("检测到mistral模型加载错误。尝试使用特殊处理...")
            # 针对mistral模型的特殊处理
            try:
                from transformers import MistralForCausalLM, MistralConfig
                
                print("使用MistralForCausalLM直接加载")
                model = MistralForCausalLM.from_pretrained(
                    model_name,
                    trust_remote_code=trust_remote_code,
                    torch_dtype=torch.float16,
                    device_map="auto" if torch.cuda.is_available() else None
                )
                tokenizer = AutoTokenizer.from_pretrained(
                    model_name,
                    trust_remote_code=trust_remote_code
                )
                model = model.to(DEVICE)
                print(f"Mistral模型加载成功: {model_name}")
            except Exception as mistral_error:
                print(f"Mistral特殊处理失败: {mistral_error}")
                raise
        else:
            raise
    
    # 构建候选池索引映射
    misconception_ids = misconception_mapping['MisconceptionId'].values.tolist()
    misconception_texts = misconception_mapping['MisconceptionName'].values.tolist()
    id_to_index = {_id: idx for idx, _id in enumerate(misconception_ids)}

    # 生成候选池向量
    def encode_texts(texts, is_query=False):
        all_vectors = []
        for i in tqdm(range(0, len(texts), PER_GPU_BATCH_SIZE)):
            batch_texts = texts[i:i+PER_GPU_BATCH_SIZE]
            
            # 添加指令前缀（如果需要）
            if is_query and model_config.get("query_instruction", ""):
                batch_texts = [model_config["query_instruction"] + t for t in batch_texts]
            
            # 准备输入
            inputs = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            ).to(DEVICE)
            
            # 提取嵌入向量
            with torch.no_grad():
                if isinstance(model, AutoModelForCausalLM.__bases__[0]) or "CausalLM" in model.__class__.__name__:
                    # 对于生成模型，使用隐藏状态作为嵌入
                    outputs = model(**inputs, output_hidden_states=True)
                    # 对于生成模型，使用最后一层隐藏状态的平均作为嵌入
                    embeddings = outputs.hidden_states[-1].mean(dim=1)
                else:
                    # 对于嵌入模型，使用标准方法
                    outputs = model(**inputs)
                    embeddings = outputs.last_hidden_state[:, 0]  # 使用CLS token
            
            # 归一化（如果需要）
            if model_config.get("normalize", False):
                embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
            
            # all_vectors.append(embeddings.cpu().numpy())
            all_vectors.append(embeddings.cpu().to(torch.float32).numpy())
        
        return np.concatenate(all_vectors, axis=0)
    
    # 编码候选池
    print("编码候选池文本...")
    sentence_embeddings = encode_texts(misconception_texts)
    
    # 编码查询
    print("编码查询文本...")
    query_texts = [item["all_text"] for _, item in train_data.iterrows()]
    ground_truth_indices = [id_to_index[item["MisconceptionId"]] for _, item in train_data.iterrows()]
    query_embeddings = encode_texts(query_texts, is_query=True)
    
    # 计算Recall@50
    def calculate_recall(sent_emb, query_emb, gt_indices, top_k=50):
        index = faiss.IndexFlatIP(sent_emb.shape[1])
        index.add(sent_emb.astype(np.float32))
        
        _, top_indices = index.search(query_emb.astype(np.float32), top_k)
        return np.mean([1 if gt in indices else 0 for gt, indices in zip(gt_indices, top_indices)])
    
    print("计算Recall@50...")
    return calculate_recall(sentence_embeddings, query_embeddings, ground_truth_indices)


# 使用示例:
# results = {}
# for model_name, config in MODELS_TO_TEST.items():
#     print(f"\n=== Evaluating {model_name} ===")
#     recall = evaluate_model(config, misconception_mapping, train_long)
#     results[model_name] = recall
#     print(f"{model_name} Recall@50: {recall*100:.2f}%")


# ======================================================
# 执行测试
# ======================================================

    # 加载数据（需替换为实际数据）
    # misconception_mapping = ... 
    # train_data = ...


results = {}
for model_name, config in MODELS_TO_TEST.items():
    print(f"\n=== Evaluating {model_name} ===")
    recall = evaluate_model(config, misconception_mapping, train_long)
    results[model_name] = recall
    print(f"{model_name} Recall@50: {recall*100:.2f}%")

# 打印最终对比结果
print("\n=== Final Results ===")
for model, score in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{model:<15} | Recall@50: {score*100:.2f}%")


=== Evaluating multilingual-e5-large ===
Loading model: intfloat/multilingual-e5-large
使用AutoModel加载嵌入模型
模型加载成功: intfloat/multilingual-e5-large
编码候选池文本...


100%|██████████| 81/81 [00:01<00:00, 42.38it/s]


编码查询文本...


100%|██████████| 137/137 [00:16<00:00,  8.51it/s]


计算Recall@50...
multilingual-e5-large Recall@50: 41.56%

=== Evaluating snowflake-arctic-embed-l-v2.0 ===
Loading model: Snowflake/snowflake-arctic-embed-l-v2.0
使用AutoModel加载嵌入模型
模型加载成功: Snowflake/snowflake-arctic-embed-l-v2.0
编码候选池文本...


100%|██████████| 81/81 [00:01<00:00, 41.37it/s]


编码查询文本...


100%|██████████| 137/137 [00:16<00:00,  8.48it/s]


计算Recall@50...
snowflake-arctic-embed-l-v2.0 Recall@50: 62.86%

=== Final Results ===
snowflake-arctic-embed-l-v2.0 | Recall@50: 62.86%
multilingual-e5-large | Recall@50: 41.56%


In [ ]:
# === Final Results ===
# BGE-large-en    | Recall@50: 64.51%
# BGE-M3          | Recall@50: 59.04%
# jina-embeddings-v3 Recall@50: 67.12%
# multilingual-e5-large Recall@50: 41.56%
# snowflake-arctic-embed-l-v2.0 | Recall@50: 62.86%

import gzip
import json
import pandas as pd

def load_json_gz_to_dataframe(file_path: str) -> pd.DataFrame:
    """
    从 .json.gz 文件中读取每行一个 JSON 对象，转换为 pandas DataFrame。

    参数:
        file_path: 压缩 JSON 文件路径

    返回:
        pandas DataFrame
    """
    data = []
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                try:
                    obj = json.loads(line)
                    data.append(obj)
                except json.JSONDecodeError:
                    continue  # 跳过格式错误的行

    return pd.DataFrame(data)

# 示例用法
df = load_json_gz_to_dataframe('../input/json_pack_full.json.gz')
print(df.head())
index=False)

  query_id  content_id                                        explanation  \
0      0_D        1672  Correct calculation: 3×(2+4)-5 = 3×6-5 = 18-5 ...   
1   1000_A         891  Correct Calculation:\n\( \frac{1-t}{t-1} = \fr...   
2   1000_D         353  Correct calculation:\n\[\n\frac{1-t}{t-1} = \f...   
3   1002_B        1715  To convert \( \frac{3}{20} \) to a decimal, th...   
4   1002_C        2308  Correct calculation: \( 3 \div 20 = 0.15 \).  ...   

                    timestamp error  
0  2025-04-20T22:00:24.440664  None  
1  2025-04-20T22:00:24.440664  None  
2  2025-04-20T22:00:24.440664  None  
3  2025-04-21T21:23:04.159306  None  
4  2025-04-20T22:00:24.440664  None  


In [71]:
import pandas as pd 

DATAPATH = '../input/synthetic.csv'
df_query = pd.read_csv(DATAPATH)  


DATAPATH = '../input/fold_df.csv'
df_fold = pd.read_csv(DATAPATH)  
df_query.head()


,query_id,content_id,MisconceptionName,SubjectName,ConstructName,QuestionText,CorrectAnswerText,InCorrectAnswerText,AllOptionText,negative_ids,related_misconceptions,Explanation
0,84_A,1568,Includes the x variable when giving the equati...,Horizontal and Vertical Lines,Recognise the equations of horizontal lines fr...,What is the equation of this line? ![A graph o...,\( y=3 \),\( y=3 x \),\r\n- \( y=3 x \)\r\n- \( y=3 \)\r\n- \( x=3 \...,"[780, 2371, 3429, 1144, 2214]",- Believes the constant only gives the y-inter...,"When seeing a horizontal line at y=3, the stud..."
1,1591_A,2450,Does not half sum of parallel sides when findi...,Volume of Prisms,Calculate the volume of other prisms when the ...,What is the area of the shaded face? ![A prism...,\( 40 \mathrm{~mm}^{2} \),\( 80 \mathrm{~mm}^{2} \),\r\n- \( 80 \mathrm{~mm}^{2} \)\r\n- \( 300 \m...,"[2332, 71, 910, 1175]",- Uses slant height instead of perpendicular h...,The correct area of a trapezium is found using...
2,885_A,1877,Thinks rectangles have rotational symmetry or ...,Properties of Quadrilaterals,Identify the order of rotational symmetry of a...,Which shape has rotational symmetry order \( 4...,![Square](),![Rectangle](),\r\n- ![Rectangle]()\r\n- ![Square]()\r\n- ![T...,"[1180, 1007, 618, 2261]",- Does not know the properties of a rectangle\...,A shape with rotational symmetry of order 4 mu...
3,59_C,2206,Added the values together instead of finding t...,Percentages of an Amount,Calculate percentage of an amount mentally whe...,\( 130 \% \) of \( 40= \),\( 52 \),\( 170 \),\r\n- \( 12 \)\r\n- \( 52 \)\r\n- \( 170 \)\r\...,"[5705, 2138, 2112, 329]",- Uses the given percentage as the result.\r\n...,The correct calculation requires multiplying 4...
4,1193_C,2362,Believes angles on a straight line add to 360 ...,"Basic Angle Facts (straight line, opposite, ar...",Find missing angles using angles on a straight...,The sum of angle \( a \) and \( b \) would be....,\( 180^{\circ} \),\( 360^{\circ} \),\r\n- \( 180^{\circ} \)\r\n- \( 100^{\circ} \)...,"[2117, 4578, 9, 4178]",- Thinks that all angles anywhere along a stra...,"When two angles form a straight line, they cre..."


In [70]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

def generate_stratified_folds(query_df, n_folds=5):
    """
    生成分层交叉验证的fold，保证：
    1. 相同original_query_id的样本在同一fold
    2. 每个fold的样本量尽可能均衡
    3. 考虑不同original_query_id的样本数量分布
    
    参数：
    query_df : DataFrame 必须包含original_query_id列
    n_folds : int fold数量，默认为5
    
    返回：
    包含fold_id的DataFrame
    """
    # 创建分组统计信息
    group_info = query_df.groupby('original_query_id').agg(
        group_size=('query_id', 'count'),
        query_ids=('query_id', list)
    ).reset_index()
    
    # 根据组大小分层
    group_size_bins = pd.cut(group_info['group_size'], 
                           bins=[0,1,2,3,np.inf],
                           labels=['1','2','3','4+'])
    group_info['strata'] = group_size_bins.astype(str)
    
    # 初始化分层KFold
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    # 为每个group分配fold
    group_info['fold_id'] = -1
    for fold, (_, test_idx) in enumerate(skf.split(X=group_info, y=group_info['strata'])):
        group_info.iloc[test_idx, group_info.columns.get_loc('fold_id')] = fold
    
    # 展开到原始query_id
    fold_map = []
    for _, row in group_info.iterrows():
        for qid in row['query_ids']:
            fold_map.append({'query_id': qid, 'fold_id': row['fold_id']})
    
    fold_df = pd.DataFrame(fold_map)
    
    # 合并到原始数据
    query_df_fold = query_df.merge(fold_df, on='query_id', how='left')
    
    return query_df_fold

# 使用示例

df_query['original_query_id'] = df_query['query_id'].str.split('_').str[0]

# 生成fold
fold_df = generate_stratified_folds(df_query)

# 验证分布
print(fold_df.groupby('fold_id').agg(
    num_queries=('query_id', 'count'),
    num_groups=('original_query_id', 'nunique'),
    avg_group_size=('original_query_id', lambda x: fold_df.groupby('original_query_id')['query_id'].count().mean())
))

ModuleNotFoundError: No module named 'sklearn'

In [72]:
fold_df

NameError: name 'fold_df' is not defined

In [67]:
df_query

,query_id,content_id,MisconceptionName,SubjectName,ConstructName,QuestionText,CorrectAnswerText,InCorrectAnswerText,AllOptionText,negative_ids,related_misconceptions,Explanation
0,84_A,1568,Includes the x variable when giving the equati...,Horizontal and Vertical Lines,Recognise the equations of horizontal lines fr...,What is the equation of this line? ![A graph o...,\( y=3 \),\( y=3 x \),\r\n- \( y=3 x \)\r\n- \( y=3 \)\r\n- \( x=3 \...,"[780, 2371, 3429, 1144, 2214]",- Believes the constant only gives the y-inter...,"When seeing a horizontal line at y=3, the stud..."
1,1591_A,2450,Does not half sum of parallel sides when findi...,Volume of Prisms,Calculate the volume of other prisms when the ...,What is the area of the shaded face? ![A prism...,\( 40 \mathrm{~mm}^{2} \),\( 80 \mathrm{~mm}^{2} \),\r\n- \( 80 \mathrm{~mm}^{2} \)\r\n- \( 300 \m...,"[2332, 71, 910, 1175]",- Uses slant height instead of perpendicular h...,The correct area of a trapezium is found using...
2,885_A,1877,Thinks rectangles have rotational symmetry or ...,Properties of Quadrilaterals,Identify the order of rotational symmetry of a...,Which shape has rotational symmetry order \( 4...,![Square](),![Rectangle](),\r\n- ![Rectangle]()\r\n- ![Square]()\r\n- ![T...,"[1180, 1007, 618, 2261]",- Does not know the properties of a rectangle\...,A shape with rotational symmetry of order 4 mu...
3,59_C,2206,Added the values together instead of finding t...,Percentages of an Amount,Calculate percentage of an amount mentally whe...,\( 130 \% \) of \( 40= \),\( 52 \),\( 170 \),\r\n- \( 12 \)\r\n- \( 52 \)\r\n- \( 170 \)\r\...,"[5705, 2138, 2112, 329]",- Uses the given percentage as the result.\r\n...,The correct calculation requires multiplying 4...
4,1193_C,2362,Believes angles on a straight line add to 360 ...,"Basic Angle Facts (straight line, opposite, ar...",Find missing angles using angles on a straight...,The sum of angle \( a \) and \( b \) would be....,\( 180^{\circ} \),\( 360^{\circ} \),\r\n- \( 180^{\circ} \)\r\n- \( 100^{\circ} \)...,"[2117, 4578, 9, 4178]",- Thinks that all angles anywhere along a stra...,"When two angles form a straight line, they cre..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2970,587_A,706,Carries out operations from right to left rega...,BIDMAS,Use the order of operations to carry out calcu...,Problem P: \( 6 \div 3+5= \)\r\nProblem Q: \( ...,Problem P: \( 6 \div 3 \)\r\nProblem Q: \( 6 \...,Problem P: \( 3+5 \)\r\nProblem Q: \( 3 \times...,\r\n- Problem P: \( 3+5 \)\r\nProblem Q: \( 3 ...,"[2532, 3524, 2306, 1507]",- Believes order of operations does not affect...,The correct solution requires performing divis...
2971,355_D,926,Believes the number of sides gives the order o...,Rotational Symmetry,Identify the order of rotational symmetry of a...,Which quadrilateral has the highest order of r...,![Parallelogram](),They each have the same order of rotational sy...,\r\n- ![Trapezium]()\r\n- ![Irregular quadrila...,"[1007, 3255, 1687, 255, 557]",- Does not connect rotational symmetry to numb...,The student incorrectly equated the number of ...
2972,983_C,234,Subtracts instead of adds,Substitution into Formula,Substitute positive integer values into expres...,If \( h=3 \) what is the value of \( 2+h \) ?,\( 5 \),\( -1 \),\r\n- \( 5 \)\r\n- \( 23 \)\r\n- \( -1 \)\r\n-...,"[3483, 1514, 2225, 3492]",- Believes the answer is the sum of the number...,The correct solution requires adding 2 and 3 (...
2973,1306_C,2275,Believes midpoint calculation is just half of ...,Midpoint Between Two Co-ordinates,Given a midpoint and a coordinate work out the...,Here are two points and their midpoint.\r\nWha...,\( \Delta=-6 \),\( \Delta=-6.5 \),\r\n- \( \Delta=-7 \)\r\n- \( \Delta=-6 \)\r\n...,"[393, 118, 2416, 761, 2427]",- Believes adding coordinates of endpoint and ...,The correct solution requires finding y₂ by us...


In [65]:
# for import_ in import_data[15:]:
#     print(import_)

In [54]:
# negatives_hard.groupby("query_id")["hard_negatives"].apply(list).to_dict()

# {'0_D': ['15,1941,158,1054,2181,2306,2488,987,561,1999'],

# 假设 negatives_hard 已经是你原始的 DataFrame
# 将 "hard_negatives" 列中的每个字符串按逗号分隔，并返回一个字典
negatives_hard_grouped = negatives_hard.groupby("query_id")["hard_negatives"].apply(lambda x: x.str.split(',').explode().tolist()).to_dict()

# 打印结果
negatives_hard_grouped

{'0_D': ['15',
  '1941',
  '158',
  '1054',
  '2181',
  '2306',
  '2488',
  '987',
  '561',
  '1999'],
 '1000_A': ['353',
  '885',
  '1562',
  '403',
  '1755',
  '167',
  '1540',
  '2398',
  '1593',
  '633'],
 '1000_D': ['891',
  '2021',
  '1469',
  '774',
  '1755',
  '167',
  '1540',
  '2398',
  '1593',
  '80'],
 '1002_B': ['2308',
  '760',
  '2242',
  '2073',
  '1542',
  '988',
  '679',
  '1424',
  '579',
  '1840'],
 '1002_C': ['1715',
  '2242',
  '760',
  '2368',
  '1542',
  '2073',
  '679',
  '1424',
  '579',
  '1840'],
 '1003_A': ['1932',
  '503',
  '2558',
  '1310',
  '1128',
  '2192',
  '1767',
  '2373',
  '561',
  '302'],
 '1004_B': ['565',
  '2326',
  '147',
  '302',
  '2238',
  '856',
  '1565',
  '561',
  '2138',
  '329'],
 '1005_B': ['2073',
  '760',
  '2242',
  '2308',
  '1637',
  '1523',
  '679',
  '1424',
  '579',
  '1840'],
 '1005_C': ['1715',
  '2242',
  '760',
  '2308',
  '1637',
  '1523',
  '679',
  '1424',
  '579',
  '1840'],
 '1006_A': ['2197',
  '305',
  '1788',
  

In [39]:
eedi_content['MisconceptionId'] = eedi_content['MisconceptionId'].astype(str)
eedi_content_dict = eedi_content.set_index('MisconceptionId')['MisconceptionName'].to_dict()

def get_content_of_hardnage_byquery_id(query_id, eedi_content_dict, negatives_hard):
    """
    根据查询ID获取相关的hardneg内容
    :param query_id: 查询ID
    :param eedi_content: eedi_content DataFrame
    :param negatives_hard: hardnegatives DataFrame
    :return: 相关的hardneg内容
    """
    # 获取与查询ID匹配的hardneg内容
    # row_to_dict
    if query_id not in negatives_hard['query_id'].values:
        print("当前的query_id在hardnegatives中不存在")
        return []
    query_row_to_dict = negatives_hard[negatives_hard['query_id'] == query_id].to_dict(orient='records')[0]
    current_content_id = query_row_to_dict['content_id']
    
    # 获取hardneg内容的ID列表
    hardneg_ids = query_row_to_dict['hard_negatives'].split(",")
    
    # 根据ID从eedi_content中获取对应的内容
    related_contents = [eedi_content_dict.get(hardneg_id.strip(), '未获取到相关内容') for hardneg_id in hardneg_ids]
    current_contents = eedi_content_dict.get(str(current_content_id), '未获取到相关内容')
    return related_contents,current_contents




ModuleNotFoundError: No module named 'sklearn'

In [74]:
# !pip install sklearn
len(eedi_content_dict)

2587

In [43]:
qid = '885_A'
related_contents,current_content = get_content_of_hardnage_byquery_id(qid, eedi_content_dict, negatives_hard)
print('当前content内容:', current_content)
for i, content in enumerate(related_contents):
    print(f"Related content {i+1}: {content}")

当前content内容: Thinks rectangles have rotational symmetry or order 4
Related content 1: Thinks rectangles have line symmetry in the diagonals 
Related content 2: Believes the number of sides gives the order of rotational symmetry
Related content 3: Believes a rectangle has four equal sides
Related content 4: Does not connect rotational symmetry to number of sides/vertices
Related content 5: Does not know how to find order of rotational symmetry
Related content 6: Misinterprets the order of division in worded problems
Related content 7: Does not know the properties of an equilateral triangle
Related content 8: Does not know the properties of an isosceles triangle
Related content 9: Does not believe a shape can have two correct names e.g. rectangle and parallelogram
Related content 10: Does not know the diagonals of a square bisect at right angles


In [ ]:
# !pip install -U huggingface_hub
# !export HF_ENDPOINT=https://hf-mirror.com

# !pip install einops
# !rm -rf ~/.cache/huggingface/transformers_modules/
